In [15]:
from cgitb import html
import click
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
import datetime as dt

# 크롤링

In [2]:
# chromedirver 불러오기
path = 'c:/PYTHON/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.maximize_window()
sleep(1)

com_l = []
title_l = []
time_l = []
text_l = []

page = range(1,17164) #카카오 맨 끝
list = [3,4,5,6,7,9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25]
st = pd.to_datetime("2022.06.30 00:00")
et = pd.to_datetime("2022.06.30 15:00")

#페이지별 전체 글 클롤링
#글 순서 특이하게 3부터 시작 한문단씩 띄어있음 주의

for j in page:

   page = "https://finance.naver.com/item/board.naver?code=035720&page=" + str(j)
   driver.get(page)
   for i in list:
      time = driver.find_element(By.XPATH, 
                                                "//*[@id='content']/div[2]/table[1]/tbody/tr[" + str(i) + "]/td[1]").text
      time = pd.to_datetime(time)

      if time >= st and time <= et:
         cl = driver.find_element(By.XPATH, 
                                          "//*[@id='content']/div[2]/table[1]/tbody/tr[" + str(i) + "]/td[2]/a").click()
         com = driver.find_element(By.XPATH, 
                                    "//*[@id='middle']/div[1]/div[1]/h2/a").text
         title = driver.find_element(By.XPATH, 
                                    "//*[@id='content']/div[2]/table[1]/tbody/tr[1]/th[1]/strong").text
         time = driver.find_element(By.XPATH, 
                                    "//*[@id='content']/div[2]/table[1]/tbody/tr[2]/th[2]").text
         text = driver.find_element(By.XPATH, 
                                       "//*[@id='body']").text


         com_l.append(com)
         title_l.append(title)
         time_l.append(time)
         text_l.append(text)

      #바로 전 페이지로 이동 메인페이지를 넣어서 안됐었음;;
         driver.execute_script("window.history.go(-1)")
         sleep(0.3)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=103.0.5060.134)


In [16]:
from operator import index
import pandas as pd
import datetime as dt

In [5]:
df = pd.DataFrame({'전체날짜' : time_l, '종목명' : com_l, '제목' : title_l, '본문' : text_l})
df.head()

NameError: name 'time_l' is not defined

## 크롤링 데이터 csv로 저장

In [ ]:
df.to_csv("./data/kakao_2206.csv")

# 종토방
## 종토방 불러오기

In [17]:
df = pd.read_csv("./data/kakao_2206.csv")
df

,전체날짜,종목명,제목,본문
0,2022.06.01 05:23,카카오,이시간에 카카오메인이 조선이 싹쓸이하네...,메인 7개 뉴스중에 5개가 조선1개가 동아 1개가 듣보잡 인터넷언론....이러니 주...
1,2022.06.01 08:44,카카오,돈은 많이 받으면서,근무시간에 오디오라인 열어놓는게 불만이라니\n그냥 집에서 누워서 tv보면서 근무하고...
2,2022.06.01 08:59,카카오,유영두 할아버지 넘 실망이에요~~^^,"토른방에 남겨진 유영두 할아버지에 대한 댓글 입니다.\n""범수 똘마니 의심 ""\n""..."
3,2022.06.01 09:29,카카오,빡빡이 하는짓 보면 모르나,김범수가 일궈낸 자리 편하게 앉아서 메타버스 빌미로 날로 먹을려고 하는거지\n\n카...
4,2022.06.01 09:37,카카오,한오백년 전쟁하자는 미필 양키괴뢰!!!,"우리는 원래 하나였으나, 미,소에 의해 강제로 분단 당했습니다.\n피눈물도 없다는 ..."
...,...,...,...,...
6615,2022.06.30 23:31,카카오,이거보고 윤석열이 왕따라고 생각하면,https://youtu.be/C2-HTUyE6Y4\n\n당신은 천재ㅋㅋㅋ 왕따 아...
6616,2022.06.30 23:34,카카오,잠이 안온다,내일 인버스에 얼마나 꽂힐지..돈복사
6617,2022.06.30 23:40,카카오,이거 액분전에 평균얼마엿어요,궁금합니다
6618,2022.06.30 23:52,카카오,나스닥 7천,간다.\n\n카카오는 3만 간다.\n\nㅋㅋ


## 날짜와 시간 분리

In [18]:
a = df.copy()
a['날짜'] = pd.to_datetime(a['전체날짜']).dt.date
a['시간'] = pd.to_datetime(a['전체날짜']).dt.time
a = a[['전체날짜', '날짜','시간', '종목명', '제목', '본문']]
a

,전체날짜,날짜,시간,종목명,제목,본문
0,2022.06.01 05:23,2022-06-01,05:23:00,카카오,이시간에 카카오메인이 조선이 싹쓸이하네...,메인 7개 뉴스중에 5개가 조선1개가 동아 1개가 듣보잡 인터넷언론....이러니 주...
1,2022.06.01 08:44,2022-06-01,08:44:00,카카오,돈은 많이 받으면서,근무시간에 오디오라인 열어놓는게 불만이라니\n그냥 집에서 누워서 tv보면서 근무하고...
2,2022.06.01 08:59,2022-06-01,08:59:00,카카오,유영두 할아버지 넘 실망이에요~~^^,"토른방에 남겨진 유영두 할아버지에 대한 댓글 입니다.\n""범수 똘마니 의심 ""\n""..."
3,2022.06.01 09:29,2022-06-01,09:29:00,카카오,빡빡이 하는짓 보면 모르나,김범수가 일궈낸 자리 편하게 앉아서 메타버스 빌미로 날로 먹을려고 하는거지\n\n카...
4,2022.06.01 09:37,2022-06-01,09:37:00,카카오,한오백년 전쟁하자는 미필 양키괴뢰!!!,"우리는 원래 하나였으나, 미,소에 의해 강제로 분단 당했습니다.\n피눈물도 없다는 ..."
...,...,...,...,...,...,...
6615,2022.06.30 23:31,2022-06-30,23:31:00,카카오,이거보고 윤석열이 왕따라고 생각하면,https://youtu.be/C2-HTUyE6Y4\n\n당신은 천재ㅋㅋㅋ 왕따 아...
6616,2022.06.30 23:34,2022-06-30,23:34:00,카카오,잠이 안온다,내일 인버스에 얼마나 꽂힐지..돈복사
6617,2022.06.30 23:40,2022-06-30,23:40:00,카카오,이거 액분전에 평균얼마엿어요,궁금합니다
6618,2022.06.30 23:52,2022-06-30,23:52:00,카카오,나스닥 7천,간다.\n\n카카오는 3만 간다.\n\nㅋㅋ


## 시간 데이터 정제

In [19]:
a['시간'] = a['시간'].astype(str)

a['시간'] = a['시간'].str.replace(":","")

a['시간'] = a['시간'].astype(int)

a['시간'] = a['시간'] / 100

a['시간'] = a['시간'].astype(int)

a

,전체날짜,날짜,시간,종목명,제목,본문
0,2022.06.01 05:23,2022-06-01,523,카카오,이시간에 카카오메인이 조선이 싹쓸이하네...,메인 7개 뉴스중에 5개가 조선1개가 동아 1개가 듣보잡 인터넷언론....이러니 주...
1,2022.06.01 08:44,2022-06-01,844,카카오,돈은 많이 받으면서,근무시간에 오디오라인 열어놓는게 불만이라니\n그냥 집에서 누워서 tv보면서 근무하고...
2,2022.06.01 08:59,2022-06-01,859,카카오,유영두 할아버지 넘 실망이에요~~^^,"토른방에 남겨진 유영두 할아버지에 대한 댓글 입니다.\n""범수 똘마니 의심 ""\n""..."
3,2022.06.01 09:29,2022-06-01,929,카카오,빡빡이 하는짓 보면 모르나,김범수가 일궈낸 자리 편하게 앉아서 메타버스 빌미로 날로 먹을려고 하는거지\n\n카...
4,2022.06.01 09:37,2022-06-01,937,카카오,한오백년 전쟁하자는 미필 양키괴뢰!!!,"우리는 원래 하나였으나, 미,소에 의해 강제로 분단 당했습니다.\n피눈물도 없다는 ..."
...,...,...,...,...,...,...
6615,2022.06.30 23:31,2022-06-30,2331,카카오,이거보고 윤석열이 왕따라고 생각하면,https://youtu.be/C2-HTUyE6Y4\n\n당신은 천재ㅋㅋㅋ 왕따 아...
6616,2022.06.30 23:34,2022-06-30,2334,카카오,잠이 안온다,내일 인버스에 얼마나 꽂힐지..돈복사
6617,2022.06.30 23:40,2022-06-30,2340,카카오,이거 액분전에 평균얼마엿어요,궁금합니다
6618,2022.06.30 23:52,2022-06-30,2352,카카오,나스닥 7천,간다.\n\n카카오는 3만 간다.\n\nㅋㅋ


In [20]:
def test(b):
    if b <= 900:
        return '09:00:00'
    if b <= 1000:
        return '10:00:00'
    if b <= 1100:
        return '11:00:00'
    if b <= 1200:
        return '12:00:00'
    if b <= 1300:
        return '13:00:00'
    if b <= 1400:
        return '14:00:00'
    if b <= 1500:
        return '15:00:00'
    if b <= 1600:
        return '16:00:00'

a['시간변환'] = a['시간'].apply(test)
a

,전체날짜,날짜,시간,종목명,제목,본문,시간변환
0,2022.06.01 05:23,2022-06-01,523,카카오,이시간에 카카오메인이 조선이 싹쓸이하네...,메인 7개 뉴스중에 5개가 조선1개가 동아 1개가 듣보잡 인터넷언론....이러니 주...,09:00:00
1,2022.06.01 08:44,2022-06-01,844,카카오,돈은 많이 받으면서,근무시간에 오디오라인 열어놓는게 불만이라니\n그냥 집에서 누워서 tv보면서 근무하고...,09:00:00
2,2022.06.01 08:59,2022-06-01,859,카카오,유영두 할아버지 넘 실망이에요~~^^,"토른방에 남겨진 유영두 할아버지에 대한 댓글 입니다.\n""범수 똘마니 의심 ""\n""...",09:00:00
3,2022.06.01 09:29,2022-06-01,929,카카오,빡빡이 하는짓 보면 모르나,김범수가 일궈낸 자리 편하게 앉아서 메타버스 빌미로 날로 먹을려고 하는거지\n\n카...,10:00:00
4,2022.06.01 09:37,2022-06-01,937,카카오,한오백년 전쟁하자는 미필 양키괴뢰!!!,"우리는 원래 하나였으나, 미,소에 의해 강제로 분단 당했습니다.\n피눈물도 없다는 ...",10:00:00
...,...,...,...,...,...,...,...
6615,2022.06.30 23:31,2022-06-30,2331,카카오,이거보고 윤석열이 왕따라고 생각하면,https://youtu.be/C2-HTUyE6Y4\n\n당신은 천재ㅋㅋㅋ 왕따 아...,None
6616,2022.06.30 23:34,2022-06-30,2334,카카오,잠이 안온다,내일 인버스에 얼마나 꽂힐지..돈복사,None
6617,2022.06.30 23:40,2022-06-30,2340,카카오,이거 액분전에 평균얼마엿어요,궁금합니다,None
6618,2022.06.30 23:52,2022-06-30,2352,카카오,나스닥 7천,간다.\n\n카카오는 3만 간다.\n\nㅋㅋ,None


In [21]:
pr = pd.read_excel("C:/PYTHON/G_ML/해커톤/kakao_price.xlsx")
pr['날짜'] = pd.to_datetime(pr['일시']).dt.date
pr['시간'] = pd.to_datetime(pr['일시']).dt.time
pr['시간'] = pr['시간'].astype(str)
pr = pr.rename(columns={'시간' : '시간변환'})

pr = pr[['날짜', '시간변환', '종가']]
pr

,날짜,시간변환,종가
0,2022-06-30,15:00:00,69900
1,2022-06-30,14:00:00,70100
2,2022-06-30,13:00:00,70100
3,2022-06-30,12:00:00,70200
4,2022-06-30,11:00:00,70400
...,...,...,...
1618,2021-07-21,13:00:00,145000
1619,2021-07-21,12:00:00,145500
1620,2021-07-21,11:00:00,145000
1621,2021-07-21,10:00:00,146500


In [22]:
a_l = pd.merge(a, pr, how='inner', on=['시간변환', '날짜'])
a_l

,전체날짜,날짜,시간,종목명,제목,본문,시간변환,종가
0,2022.06.02 01:45,2022-06-02,145,카카오,내가 말했재??,카카오는 너무 씹거품이라고 네이버사라고 ㅋㅋㅋ\n\n네이버는 살만하다\n\n난 둘다...,09:00:00,83300
1,2022.06.02 01:54,2022-06-02,154,카카오,카카오 주가 전망 및 분석,ddomi.tistory.com/43\n\n객관적인 분석일뿐\n\n감정이입은 하지 마세요,09:00:00,83300
2,2022.06.02 08:06,2022-06-02,806,카카오,"오늘 압축 영화의 재미는,,,,//",이번 선거는 이재명 죽이기\n\n경기도 이재명 흔적을 완전히 없애기\n국힘작전 이었...,09:00:00,83300
3,2022.06.02 08:17,2022-06-02,817,카카오,"성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.","성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.",09:00:00,83300
4,2022.06.02 08:19,2022-06-02,819,카카오,두번째 물어봅니다...평단 9만원 1000주 피가 마릅니다....,오나요?ㅠ,09:00:00,83300
...,...,...,...,...,...,...,...,...
4565,2022.06.30 14:48,2022-06-30,1448,카카오,가즈아 육카오,육카오 한번 들렸다가자 칠카오,15:00:00,69900
4566,2022.06.30 14:51,2022-06-30,1451,카카오,카카오 액면분할 성공적이네요,3분의 1 액면분할 성공적으로 진행중이네요\n축하드립니다 주주분들,15:00:00,69900
4567,2022.06.30 14:53,2022-06-30,1453,카카오,현금 3억 대기중,카카오 3만에서 현금 3억 대기중,15:00:00,69900
4568,2022.06.30 14:54,2022-06-30,1454,카카오,외쿡쉐키들,과연 뭐하는 짓일까요 막아주는척 쑈 하는걸까요,15:00:00,69900


In [25]:
from konlpy.tag import Okt
import re

In [26]:
okt = Okt()
a_s = a_l.loc[a_l['시간변환']=='09:00:00']
a_s


,전체날짜,날짜,시간,종목명,제목,본문,시간변환,종가
0,2022.06.02 01:45,2022-06-02,145,카카오,내가 말했재??,카카오는 너무 씹거품이라고 네이버사라고 ㅋㅋㅋ\n\n네이버는 살만하다\n\n난 둘다...,09:00:00,83300
1,2022.06.02 01:54,2022-06-02,154,카카오,카카오 주가 전망 및 분석,ddomi.tistory.com/43\n\n객관적인 분석일뿐\n\n감정이입은 하지 마세요,09:00:00,83300
2,2022.06.02 08:06,2022-06-02,806,카카오,"오늘 압축 영화의 재미는,,,,//",이번 선거는 이재명 죽이기\n\n경기도 이재명 흔적을 완전히 없애기\n국힘작전 이었...,09:00:00,83300
3,2022.06.02 08:17,2022-06-02,817,카카오,"성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.","성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.",09:00:00,83300
4,2022.06.02 08:19,2022-06-02,819,카카오,두번째 물어봅니다...평단 9만원 1000주 피가 마릅니다....,오나요?ㅠ,09:00:00,83300
...,...,...,...,...,...,...,...,...
4491,2022.06.30 08:16,2022-06-30,816,카카오,까톡 까톡,종가 60000 까톡,09:00:00,70400
4492,2022.06.30 08:23,2022-06-30,823,카카오,"성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.","성탄절이 토요일, 일요일일 경우 월요일을 대체공휴일로 지정해주세요.",09:00:00,70400
4493,2022.06.30 08:27,2022-06-30,827,카카오,코로나가 개미들 다 죽이겠네,버티면 올라온다는 되도 않는 망상을 심어줬네\n\n코로나 질병으로 디진 놈들 보다\...,09:00:00,70400
4494,2022.06.30 08:30,2022-06-30,830,카카오,카카오 = 꼬끼오,즉 후라이드 닭한마리값\n2만. 적정선.,09:00:00,70400


In [27]:
text = Okt.nouns(a_s['본문'].to_string())
text

TypeError: nouns() missing 1 required positional argument: 'phrase'

In [281]:
from collections import Counter

count = Counter(text)
tag_count = []
tags = []

for n, c in count.most_common(100):
    dics = {'tag': n, 'count': c}
    if len(dics['tag']) >= 2 and len(tags) <= 49:
        tag_count.append(dics)
        tags.append(dics['tag'])

for tag in tag_count:
    print(" {:<14}".format(tag['tag']), end='\t')
    print("{}".format(tag['count']))

 카카오           	3
 외인            	3
 주주            	2
 카오            	2
 지금            	2
 댓글            	2
 엊그제           	1
 신용            	1
 미수            	1
 순간            	1
 우리            	1
 대출            	1
 걸까요           	1
 현금            	1
 대기            	1
 면분            	1
 성공            	1
 진행            	1
 축하            	1
 반등            	1
 주식            	1
 평단            	1
 손해            	1
 종토            	1
 방서            	1
 욕지거리          	1
 난리            	1
 삭제            	1
 폭동            	1
 그건            	1
 회복            	1
 바닥            	1
 근처            	1
 세계            	1
 초콜렛           	1
